In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math
import os
from scipy.optimize import brentq
from sklearn.neighbors import KernelDensity
import xarray as xr

from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import KalmanFilter as KF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.common import Q_continuous_white_noise

from pumahu import Forwardmodel, ns_sampling, LakeData, get_data
from pumahu.visualise import trellis_plot, mcmc_heat_input, heat_vs_rsam

In [ ]:
def xr2xr(xdfin):
    """
    Helper function to convert old netcdf file structure to new
    """
    new_exp = np.zeros((xdfin['exp'].shape[0], xdfin['exp'].shape[1], 2))
    new_exp[:, :, 0] = xdfin['exp'].values[:]
    new_exp[:, :, 1] = xdfin['var'].values[:]

    return xr.Dataset({'exp': (('dates', 'parameters', 'val_std'), new_exp)},
                      {'dates': xdfin.dates.values,
                       'parameters': xdfin.parameters.values,
                       'val_std': ['val', 'std']})

## Table of contents
---------------------------
1. [Inversion](#Inversion)
2. [Forecasting](#Forecasting)

## Inversion

In [ ]:
startdate = '2020-12-01'
enddate = '2021-01-11'
res_dir = './data'
if not os.path.isdir(res_dir):
    os.makedirs(res_dir)
mout_prior = get_data('./data/outflow_prior.npz')

In [ ]:
ld = LakeData()
data = ld.get_data(startdate, enddate)
# Setup path for results file
tstart = pd.to_datetime(data['dates'].values[0])
tend = pd.to_datetime(data['dates'].values[-1])
res_fn = 'mcmc_sampling_{:s}_{:s}.nc'
res_fn = res_fn.format(tstart.strftime('%Y-%m-%d'),
                       tend.strftime('%Y-%m-%d'))
res_fn = os.path.join(res_dir, res_fn)

ns_sampling(data, res_fn, nsamples=1001, nresample=500,
            q_in_min=0., q_in_max=1000., m_in_min=0., m_in_max=20.,
            m_out_min=0., m_out_max=20., new=True,
            m_out_prior=mout_prior, tolZ=1e-3, lh_fun=None,
            tolH=3., H=6., ws=4.5, seed=-1, intmethod='euler',
            gradient=False)


In [ ]:
xdf = xr.open_dataset(res_fn)

In [ ]:
_ = mcmc_heat_input(xdf, filename='heat.png')

In [ ]:
_ = heat_vs_rsam(xdf)

In [ ]:
#benchmark_file = './data/benchmarking/forward_2019-01-01_2021-01-11.nc'
benchmark_file = None
xdft = None
if benchmark_file is not None:
    xdft = xr2xr(xr.open_dataset(benchmark_file))

In [ ]:
trellis_plot(xdf, data2=xdft)